**Lecturer:**  
**Group: 228 (235## Heading)**       
**Lab User:**  

**Student: Benjamin Purchase**     **NIA: 100557337**  
**Student: Ishaan Kalra**     **NIA: 100559793**  
**Student Lucas Pedemonte:**     **NIA: 100558486**  


# Introduction

It will consist of an introductory paragraph, dealing with the problem to be solved, analyzing it and fixing the goals of the work. It will also include the SQL code generated for this *labwork* (*queries*, *views,* *triggers*, ...). Finally, it should describe the document structure.


# Queries

#### **1.1 a) BoreBooks: Books with Editions in ≥3 Languages and No Loans**  

```
1. BookEditions := π(title, author, language) (books ⋈ editions)
2. LanguageCount := γ(title, author); COUNT(DISTINCT language)→lang_count (BookEditions)
3. QualifiedBooks := σ(lang_count ≥ 3) (LanguageCount)
4. LoanedBooks := π(title, author) (loans ⋈ copies ⋈ editions ⋈ books)
5. BoreBooks := QualifiedBooks - LoanedBooks
```

##### **Key operators:**
- **⋈** : Natural join (combines tables by matching column names)
- **π** : Projection (selects specific columns)
- **γ** : Group-by aggregation
- **σ** : Selection (filters rows)
- **-** : Set difference (finds records in first set but not second)

##### **How it works**:
1. First joins books with their editions
2. Groups by book + author to count distinct languages
3. Filters for books with ≥3 language editions
4. Finds all books that have been loaned
5. Returns only books that qualify (≥3 languages that haven't been loaned)

##### **SQL Implementation**  
```sql
WITH BookEditions AS (
    SELECT b.title, b.author, e.language, e.isbn
    FROM books b
    JOIN editions e ON b.title = e.title AND b.author = e.author
),
LanguageCount AS (
    SELECT title, author
    FROM BookEditions
    GROUP BY title, author
    HAVING COUNT(DISTINCT language) >= 3
),
LoanedBooks AS (
    SELECT DISTINCT b.title, b.author
    FROM copies c
    JOIN loans l ON c.signature = l.signature
    JOIN editions e ON c.isbn = e.isbn
    JOIN books b ON e.title = b.title AND e.author = b.author
)
SELECT lc.title, lc.author
FROM LanguageCount lc
LEFT JOIN LoanedBooks lb ON lc.title = lb.title AND lc.author = lb.author
WHERE lb.title IS NULL;
```

##### **Testing**  
1. **Test Case 1**:  
   - **Scenario**: Insert a book (*"Cézanne"* by *Cézanne, Paul*) with editions in Spanish, English, and French. No copies are loaned.  
   - **Expected Result**: The book appears in the output.  
   - **Actual Result**: *"Cézanne"* is listed in the query output (first row).  

2. **Test Case 2**:  
   - **Scenario**: A book (*"Costa Brava"* by *CampaÃ±Ã¡, Antonio*) with editions in 3 languages, but three copies are loaned.  
   - **Expected Result**: The book is excluded.  
   - **Actual Result**: *"Costa Brava"* does **not** appear in the output.  

3. **Test Case 3**:  
   - **Scenario**: A book (*"Abelardo Morell"* by *Morell, Abelardo*) has editions in only 2 languages with no copies loaned.  
   - **Expected Result**: Not included.  
   - **Actual Result**: *"Abelardo Morell"* is **not** in the output.  

This shows that the query correctly filters only the books with ≥3 languages and no loans.  

---

#### **1.1 b) Reports on Employees: Driver Metrics**

##### **Relational Algebra**

```
1. DriverInfo := π(passport, fullname, birthdate, cont_start, cont_end) (drivers)
2. Seniority := γ(passport); FLOOR(MONTHS_BETWEEN(cont_end, cont_start)/12)→seniority (DriverInfo)
3. ActiveYears := γ(passport); COUNT(DISTINCT YEAR(taskdate))→active_years (assign_drv)
4. StopsPerDriver := γ(passport); COUNT(town)→total_stops (assign_drv ⋈ stops)
5. LoansPerDriver := γ(passport);
                      COUNT(signature)→total_loans,
                      SUM(CASE WHEN return IS NULL THEN 1 ELSE 0)→unreturned (services ⋈ loans)
6. FinalReport := DriverInfo ⋈ Seniority ⋈ ActiveYears ⋈ StopsPerDriver ⋈ LoansPerDriver
```

##### \*\*Key steps:

1. **DriverInfo**: Gets driver's personal data
2. **Seniority**: Calculates contracted years (FLOOR rounds down)
3. **ActiveYears**: Counts distinct years with assignments
4. **StopsPerDriver**: Counts all stops made by each driver
5. **LoansPerDriver**: Calculates:
   - Total loans handled
   - Count of unreturned loans (where return IS NULL)
6. **FinalReport**: Joins all metrics together

##### **Why it works**:

- Each γ (gamma) operation groups by passport and calculates one metric
- The final join combines all metrics for the complete report
- Handles edge cases (like division by zero) through SQL implementation
- Matches your output showing drivers with their:
  - Age (from birthdate)
  - Seniority (contract duration)
  - Stops/Year (total_stops/active_years)
  - % Unreturned (unreturned/total_loans)

This explains why see values like the following are shown:

- "109 Stops/Year" (109 stops ÷ 1 active year)
- "0% Unreturned" (when unreturned=0) in the results

##### **SQL Implementation**

```sql
WITH DriverInfo AS (
    SELECT
        passport,
        fullname,
        birthdate,
        cont_start,
        COALESCE(cont_end, SYSDATE) AS cont_end
    FROM drivers
),
Seniority AS (
    SELECT
        passport,
        FLOOR(MONTHS_BETWEEN(cont_end, cont_start) / 12) AS seniority
    FROM DriverInfo
),
ActiveYears AS (
    SELECT
        passport,
        COUNT(DISTINCT EXTRACT(YEAR FROM taskdate)) AS active_years
    FROM assign_drv
    GROUP BY passport
),
StopsPerDriver AS (
    SELECT
        ad.passport,
        COUNT(s.town) AS total_stops
    FROM assign_drv ad
    JOIN stops s ON ad.route_id = s.route_id
    GROUP BY ad.passport
),
LoansPerDriver AS (
    SELECT
        s.passport,
        COUNT(l.signature) AS total_loans,
        SUM(CASE WHEN l.return IS NULL THEN 1 ELSE 0 END) AS unreturned
    FROM services s
    JOIN loans l ON s.town = l.town AND s.province = l.province AND s.taskdate = l.stopdate
    GROUP BY s.passport
)
SELECT
    d.fullname AS "Full Name",
    FLOOR(MONTHS_BETWEEN(SYSDATE, d.birthdate) / 12) AS "Age",
    s.seniority AS "Seniority (Years)",
    ay.active_years AS "Active Years",
    NVL(sp.total_stops / NULLIF(ay.active_years, 0), 0) AS "Stops/Year",
    NVL(lp.total_loans / NULLIF(ay.active_years, 0), 0) AS "Loans/Year",
    NVL((lp.unreturned / NULLIF(lp.total_loans, 0)) * 100, 0) AS "% Unreturned"
FROM DriverInfo d
JOIN Seniority s ON d.passport = s.passport
JOIN ActiveYears ay ON d.passport = ay.passport
LEFT JOIN StopsPerDriver sp ON d.passport = sp.passport
LEFT JOIN LoansPerDriver lp ON d.passport = lp.passport;
```

##### **Testing**

1. **Test Case 1**:

   - **Scenario**: Driver _"Victoria Rojo Blanco"_ has:
     - Age: 38, Seniority: 2 years, Active Years: 1
     - Total Stops: 109, Total Loans: 1664, Unreturned: 0
   - **Expected**:
     - `Stops/Year = 109`, `Loans/Year = 1664`, `% Unreturned = 0`
   - **Actual Result**: Matches output.

2. **Test Case 2**:

   - **Scenario**: Driver _"César Pareja Feliz"_ has:
     - Active Years: 3, Total Loans: 0
   - **Expected**:
     - `Loans/Year = 0`, `% Unreturned = 0`
   - **Actual Result**: `Loans/Year = 0` (as shown in the output).

3. **Test Case 3**:
   - **Edge Case**: Driver with `active_years = 0` (e.g., no assignments).
   - **Expected**: `Stops/Year = 0`, `Loans/Year = 0` (handled by `NULLIF`).
   - **Result**: No such driver in the database.

This shows that the query accurately calculates driver metrics, handles division by zero, and aligns with the provided results.

---



# Package

## Introduction

The `FOUNDICU` package is designed to manage loans and reservations in the FSDB (Flying Squirrel Database), a mobile library system where buses stop at different municipalities to allow users to borrow and return books. The package supports crucial operations on the `loans`, `users`, `copies`, `editions`, and `services` tables to ensure system consistency.

The package contains the following procedures:

- `insert_loan`
- `insert_reservation`
- `record_return`

Each procedure checks for essential constraints like user existence, reservation or loan limits, copy availability, and service presence before proceeding with data manipulations.

## Package Structure

```sql
CREATE OR REPLACE PACKAGE foundicu AS
    PROCEDURE insert_loan(p_signature IN copies.signature%TYPE);
    PROCEDURE insert_reservation(p_isbn IN editions.isbn%TYPE, p_date IN DATE);
    PROCEDURE record_return(p_signature IN copies.signature%TYPE);
END foundicu;
/

CREATE OR REPLACE PACKAGE BODY foundicu AS
    -- Implementation of procedures below
END foundicu;
/
```

## Procedures Implementation

### Insert Loan Procedure

- Design:
  - Inputs:
    - `p_signature`: The signature of the copy to be loaned.
  - Outputs:
    - Success message or appropriate error messages.
  - Logic:
    - Check if the user exists.
    - Check if the user has a reservation for the copy on the current date.
    - If yes, convert the reservation to a loan.
    - If not:
      - Check if the copy is available for the next two weeks.
      - Check if the user has not exceeded their loan limit.
      - Check if the user is not banned.
      - Find today's service stop and create a loan.
- Implementation

  ```sql

  PROCEDURE insert_loan(p_signature IN copies.signature%TYPE) IS
  v_user_id users.user_id%TYPE;
  v_user_exists NUMBER;
  v_has_reservation NUMBER;
  v_town stops.town%TYPE;
  v_province stops.province%TYPE;
  v_stopdate DATE;
  v_copy_available NUMBER;
  v_loan_count NUMBER;
  v_is_banned NUMBER;
  v_max_loans NUMBER := 5;
  BEGIN
  v_user_id := SYS_CONTEXT('USER_CONTEXT', 'USER_ID');

      SELECT COUNT(*) INTO v_user_exists
      FROM users
      WHERE user_id = v_user_id;

      IF v_user_exists = 0 THEN
          DBMS_OUTPUT.PUT_LINE('Error: User does not exist in the system.');
          RETURN;
      END IF;

      SELECT COUNT(*), MAX(town), MAX(province), MAX(stopdate)
      INTO v_has_reservation, v_town, v_province, v_stopdate
      FROM loans
      WHERE signature = p_signature
      AND user_id = v_user_id
      AND TRUNC(SYSDATE) = TRUNC(stopdate)
      AND return IS NULL
      AND type = 'R';

      IF v_has_reservation > 0 THEN
          UPDATE loans
          SET type = 'L',
              time = 0
          WHERE signature = p_signature
          AND user_id = v_user_id
          AND TRUNC(stopdate) = TRUNC(SYSDATE)
          AND type = 'R';

          DBMS_OUTPUT.PUT_LINE('Reservation converted to loan successfully.');
      ELSE
          SELECT COUNT(*)
          INTO v_copy_available
          FROM copies
          WHERE signature = p_signature
          AND deregistered IS NULL
          AND NOT EXISTS (
              SELECT 1
              FROM loans
              WHERE signature = p_signature
                  AND return IS NULL
                  AND TRUNC(stopdate) BETWEEN TRUNC(SYSDATE) AND TRUNC(SYSDATE) + 14
          );

          IF v_copy_available = 0 THEN
              DBMS_OUTPUT.PUT_LINE('Error: Copy is not available for the next two weeks.');
              RETURN;
          END IF;

          SELECT COUNT(*)
          INTO v_loan_count
          FROM loans
          WHERE user_id = v_user_id
          AND type = 'L'
          AND return IS NULL;

          IF v_loan_count >= v_max_loans THEN
              DBMS_OUTPUT.PUT_LINE('Error: User has reached the maximum number of loans.');
              RETURN;
          END IF;

          SELECT COUNT(*)
          INTO v_is_banned
          FROM users
          WHERE user_id = v_user_id
          AND ban_up2 IS NOT NULL
          AND ban_up2 >= SYSDATE;

          IF v_is_banned > 0 THEN
              DBMS_OUTPUT.PUT_LINE('Error: User is currently banned from borrowing.');
              RETURN;
          END IF;

          BEGIN
              SELECT town, province, taskdate
              INTO v_town, v_province, v_stopdate
              FROM services
              WHERE TRUNC(taskdate) = TRUNC(SYSDATE)
              AND ROWNUM = 1;
          EXCEPTION
              WHEN NO_DATA_FOUND THEN
                  DBMS_OUTPUT.PUT_LINE('Error: No service available today.');
                  RETURN;
          END;

          INSERT INTO loans (signature, user_id, stopdate, town, province, type, time, return)
          VALUES (p_signature, v_user_id, v_stopdate, v_town, v_province, 'L', 0, NULL);

          DBMS_OUTPUT.PUT_LINE('Loan created successfully.');
      END IF;

      COMMIT;

  EXCEPTION
  WHEN OTHERS THEN
  ROLLBACK;
  DBMS_OUTPUT.PUT_LINE('Error: ' || SQLERRM);
  END insert_loan;

  ```

- Tests

  ```sql
  -- Test 1
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'NONEXIST1');
  foundicu.insert_loan('A1234'); -- Error: User does not exist in the system.

  -- Test 2
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER001');
  foundicu.insert_loan('B5678'); -- Reservation converted to loan successfully.

  -- Test 3
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER002');
  foundicu.insert_loan('C9012'); -- Error: Copy is not available for the next two weeks.

  -- Test 4
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER003');
  foundicu.insert_loan('D3456'); -- Error: User has reached the maximum number of loans.

  -- Test 5
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER004');
  foundicu.insert_loan('E7890'); -- Error: User is currently banned from borrowing.

  -- Test 6
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER005');
  foundicu.insert_loan('F1234'); -- Loan created successfully.
  ```

### Insert Reservation Procedure

- Design:
  - Inputs:
  - p_isbn: The ISBN of the book edition.
  - p_date: Desired reservation date.
  - Outputs:
  - Success or error messages.
  - Logic:
  - Validate user existence.
  - Ensure the reservation limit isn't reached.
  - Confirm the user isn't banned.
  - Check service availability on the given date.
  - Find an available copy.
  - Reserve it with the current service info.
- Implementation:

  ```sql

      PROCEDURE insert_reservation(p_isbn IN editions.isbn%TYPE, p_date IN DATE) IS
      v_user_id users.user_id%TYPE;
      v_user_exists NUMBER;
      v_reservation_count NUMBER;
      v_is_banned NUMBER;
      v_max_reservations NUMBER := 3;
      v_town stops.town%TYPE;
      v_province stops.province%TYPE;
      v_service_date DATE;
      v_available_signature copies.signature%TYPE;
      v_service_exists NUMBER;
  BEGIN
      v_user_id := SYS_CONTEXT('USER_CONTEXT', 'USER_ID');

      SELECT COUNT(*) INTO v_user_exists FROM users WHERE user_id = v_user_id;
      IF v_user_exists = 0 THEN
          DBMS_OUTPUT.PUT_LINE('Error: User does not exist in the system.');
          RETURN;
      END IF;

      SELECT COUNT(*) INTO v_reservation_count
      FROM loans WHERE user_id = v_user_id AND type = 'R' AND stopdate >= SYSDATE;

      IF v_reservation_count >= v_max_reservations THEN
          DBMS_OUTPUT.PUT_LINE('Error: User has reached the maximum number of reservations.');
          RETURN;
      END IF;

      SELECT COUNT(*) INTO v_is_banned
      FROM users WHERE user_id = v_user_id AND ban_up2 IS NOT NULL AND ban_up2 >= SYSDATE;

      IF v_is_banned > 0 THEN
          DBMS_OUTPUT.PUT_LINE('Error: User is currently banned from borrowing.');
          RETURN;
      END IF;

      SELECT COUNT(*) INTO v_service_exists
      FROM services WHERE TRUNC(taskdate) = TRUNC(p_date);
      IF v_service_exists = 0 THEN
          DBMS_OUTPUT.PUT_LINE('Error: No service available on the specified date.');
          RETURN;
      END IF;

      BEGIN
          SELECT c.signature
          INTO v_available_signature
          FROM copies c
          JOIN editions e ON c.isbn = e.isbn
          WHERE e.isbn = p_isbn
          AND c.deregistered IS NULL
          AND NOT EXISTS (
              SELECT 1 FROM loans l
              WHERE l.signature = c.signature
                  AND l.return IS NULL
                  AND l.stopdate BETWEEN TRUNC(p_date) AND TRUNC(p_date) + 14
          )
          AND ROWNUM = 1;
      EXCEPTION
          WHEN NO_DATA_FOUND THEN
              DBMS_OUTPUT.PUT_LINE('Error: No available copy of this edition for the specified date range.');
              RETURN;
      END;

      SELECT town, province, taskdate
      INTO v_town, v_province, v_service_date
      FROM services
      WHERE TRUNC(taskdate) = TRUNC(p_date)
      AND ROWNUM = 1;

      INSERT INTO loans (signature, user_id, stopdate, town, province, type, time, return)
      VALUES (v_available_signature, v_user_id, v_service_date, v_town, v_province, 'R', 0, NULL);

      DBMS_OUTPUT.PUT_LINE('Reservation created successfully for copy ' || v_available_signature || ' on ' || TO_CHAR(p_date, 'DD-MM-YYYY'));
      COMMIT;
  EXCEPTION
      WHEN OTHERS THEN
          ROLLBACK;
          DBMS_OUTPUT.PUT_LINE('Error: ' || SQLERRM);
  END insert_reservation;

  ```

- Tests:

  ```sql
        -- Test 1
    DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'NONEXIST1');
    foundicu.insert_reservation('978-3-16-148410-0', SYSDATE + 7); -- Error: User does not exist

    -- Test 2
    DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER006');
    foundicu.insert_reservation('978-3-16-148410-0', SYSDATE + 7); -- Error: Max reservations

    -- Test 3
    DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER004');
    foundicu.insert_reservation('978-3-16-148410-0', SYSDATE + 7); -- Error: Banned

    -- Test 4
    DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER007');
    foundicu.insert_reservation('978-3-16-148410-0', TO_DATE('31-12-2099', 'DD-MM-YYYY')); -- No service

    -- Test 5
    foundicu.insert_reservation('978-0-00-000000-0', SYSDATE + 7); -- No available copy

    -- Test 6
    foundicu.insert_reservation('978-3-16-148410-0', SYSDATE + 7); -- Success

  ```

### Record Books Returning Procedure

- Design
  -Inputs:
  - `p_signature`: The signature of the book being returned.
- Outputs:
  - Success or error message.
- Logic:
  - Verify if the user has a valid unreturned loan.
  - If yes, mark it returned.
  - If no, show an error.
- Implementation

  ```sql
  PROCEDURE record_return(p_signature IN copies.signature%TYPE) IS
      v_user_id users.user_id%TYPE;
      v_loan_exists NUMBER;
  BEGIN
      v_user_id := SYS_CONTEXT('USER_CONTEXT', 'USER_ID');

      SELECT COUNT(*)
      INTO v_loan_exists
      FROM loans
      WHERE signature = p_signature
      AND user_id = v_user_id
      AND type = 'L'
      AND return IS NULL;

      IF v_loan_exists > 0 THEN
          UPDATE loans
          SET return = SYSDATE
          WHERE signature = p_signature
          AND user_id = v_user_id
          AND type = 'L'
          AND return IS NULL;

          DBMS_OUTPUT.PUT_LINE('Book return recorded successfully.');
      ELSE
          DBMS_OUTPUT.PUT_LINE('Error: User has not borrowed this book or it has already been returned.');
      END IF;

      COMMIT;
  EXCEPTION
      WHEN OTHERS THEN
          ROLLBACK;
          DBMS_OUTPUT.PUT_LINE('Error: ' || SQLERRM);
  END record_return;

  ```

- Tests

  ```sql
  -- Test 1
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER008');
  foundicu.record_return('G5678'); -- Error: Book not borrowed or already returned

  -- Test 2
  DBMS_SESSION.SET_CONTEXT('USER_CONTEXT', 'USER_ID', 'USER009');
  foundicu.record_return('H9012'); -- Success

  ```

- Tests Output:
  ```sql
  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  1 row created.

  Commit complete.

  === Test 1.1: Successful Loan Creation ===
  Processing loan for user: USER000001, signature: SIG01
  Loan created successfully.

  PL/SQL procedure successfully completed.

  === Test 1.2: Loan for Non-existent User ===
  Processing loan for user: NONEXISTENT, signature: SIG01
  Error: User does not exist in the system.

  PL/SQL procedure successfully completed.

  === Test 1.3: Loan for Unavailable Copy ===
  Processing loan for user: USER000001, signature: SIG02
  Loan created successfully.
  Processing loan for user: USER000003, signature: SIG02
  Error: Copy is not available for the next two weeks.

  PL/SQL procedure successfully completed.

  === Test 1.4: Loan Limit Exceeded ===
  Processing loan for user: USER000001, signature: SIG03
  Loan created successfully.
  Processing loan for user: USER000001, signature: SIG04
  Loan created successfully.
  Processing loan for user: USER000001, signature: SIG05
  Loan created successfully.
  Processing loan for user: USER000001, signature: SIG06
  Error: User has reached the maximum number of loans.
  Processing loan for user: USER000001, signature: SIG07
  Error: User has reached the maximum number of loans.
  Processing loan for user: USER000001, signature: SIG08
  Error: User has reached the maximum number of loans.

  PL/SQL procedure successfully completed.

  === Test 1.5: Loan by Banned User ===
  Processing loan for user: USER000002, signature: SIG01
  Error: User does not exist in the system.

  PL/SQL procedure successfully completed.

  === Test 1.6: Convert Reservation to Loan ===
  Processing reservation for user: USER000001, ISBN: ISBN001, Date: 04-04-2025
  Reservation created successfully for copy SIG06 on 04-04-2025
  Processing loan for user: USER000001, signature: SIG01
  Error: Copy is not available for the next two weeks.

  PL/SQL procedure successfully completed.

  === Test 2.1: Successful Reservation Creation ===
  Processing reservation for user: USER000001, ISBN: ISBN001, Date: 05-04-2025
  Reservation created successfully for copy SIG01 on 05-04-2025

  PL/SQL procedure successfully completed.

  === Test 2.2: Reservation for Non-existent User ===
  Processing reservation for user: NONEXISTENT, ISBN: ISBN001, Date: 05-04-2025
  Error: User does not exist in the system.

  PL/SQL procedure successfully completed.

  === Test 2.3: Reservation Limit Exceeded ===
  Processing reservation for user: USER000003, ISBN: ISBN001, Date: 05-04-2025
  Reservation created successfully for copy SIG02 on 05-04-2025
  Processing reservation for user: USER000003, ISBN: ISBN001, Date: 06-04-2025
  Error: No service available on the specified date.
  Processing reservation for user: USER000003, ISBN: ISBN002, Date: 05-04-2025
  Reservation created successfully for copy SIG07 on 05-04-2025
  Processing reservation for user: USER000003, ISBN: ISBN002, Date: 06-04-2025
  Error: No service available on the specified date.

  PL/SQL procedure successfully completed.

  === Test 2.4: Reservation by Banned User ===
  Processing reservation for user: USER000002, ISBN: ISBN001, Date: 05-04-2025
  Error: User does not exist in the system.

  PL/SQL procedure successfully completed.

  === Test 2.5: Reservation with No Available Copies ===
  Processing reservation for user: USER000001, ISBN: ISBN001, Date: 07-04-2025
  Error: No service available on the specified date.
  Processing reservation for user: USER000003, ISBN: ISBN001, Date: 07-04-2025
  Error: No service available on the specified date.
  Processing reservation for user: USER000003, ISBN: ISBN001, Date: 07-04-2025
  Error: No service available on the specified date.

  PL/SQL procedure successfully completed.

  === Test 2.6: Reservation with No Service Available ===
  Processing reservation for user: USER000001, ISBN: ISBN001, Date: 13-07-2025
  Error: No service available on the specified date.

  PL/SQL procedure successfully completed.

  === Test 3.1: Successful Return Recording ===
  Processing loan for user: USER000001, signature: SIG01
  Error: Copy is not available for the next two weeks.
  Processing return for user: USER000001, signature: SIG01
  Book return recorded successfully.

  PL/SQL procedure successfully completed.

  === Test 3.2: Return for Non-existent User ===
  Processing return for user: NONEXISTENT, signature: SIG01
  Error: User has not borrowed this book or it has already been returned.

  PL/SQL procedure successfully completed.

  === Test 3.3: Return for Book Not Borrowed ===
  Processing return for user: USER000001, signature: SIG08
  Error: User has not borrowed this book or it has already been returned.

  PL/SQL procedure successfully completed.

  === Test 3.4: Return for Already Returned Book ===
  Processing loan for user: USER000001, signature: SIG01
  Error: Copy is not available for the next two weeks.
  Processing return for user: USER000001, signature: SIG01
  Error: User has not borrowed this book or it has already been returned.
  Processing return for user: USER000001, signature: SIG01
  Error: User has not borrowed this book or it has already been returned.

  PL/SQL procedure successfully completed.
  
  ```
---



# External Design

This section focuses on the **External Design** of the bibuses services user profile system. The concept of **"current user"** is resolved via a session context variable using `SYS_CONTEXT`. Three distinct views are developed:

- **`my_data`**: A read-only view that shows personal information of the current user.
- **`my_loans`**: A view showing loan history and user-submitted posts; only post texts can be updated.
- **`my_reservations`**: A view to manage reservations by the current user, supporting insertions, deletions, and updates if book availability is satisfied.

Each view is designed, implemented in SQL, and tested for correctness and operational constraints, with INSTEAD OF triggers where native view support is insufficient.

## Views Implementation:

### *Using a Package Variable (with Setter and Getter) for the Views below:*

#### Package Specification (foundicu_user.pks):
```sql
CREATE OR REPLACE PACKAGE foundicu_user IS
  current_user CHAR(10);
  PROCEDURE set_current_user(p_user CHAR);
  FUNCTION get_current_user RETURN CHAR;
END foundicu_user;
/
```
#### Package Body (foundicu_user.pkb):
```sql
CREATE OR REPLACE PACKAGE BODY foundicu_user IS
  PROCEDURE set_current_user(p_user CHAR) IS
  BEGIN
    current_user := p_user;
  END set_current_user;
  
  FUNCTION get_current_user RETURN CHAR IS
  BEGIN
    RETURN current_user;
  END get_current_user;
END foundicu_user;
/
```

### **my_data View**

#### a) Design (Relational Algebra)

$$
\pi_{\text{user\_id, id\_card, name, surname1, surname2, birthdate, town, province, address, email, phone, type}} (\sigma_{\text{user\_id} = \text{current\_user}} (\text{users}))
$$

#### b) Implementation (SQL)
```sql
CREATE OR REPLACE VIEW my_data AS
  SELECT user_id,
         id_card,
         name,
         surname1,
         surname2,
         birthdate,
         town,
         province,
         address,
         email,
         phone,
         type
  FROM users
  WHERE user_id = foundicu_user.get_current_user();
```
#### c) Tests
- Test Case 1:
```sql
EXEC foundicu_user.set_current_user('9994309942');
SELECT * FROM my_data;
```
- Works as expected, the output are the personal details of the user: 9994309942.
- <img src="1.png" height="250" width="800">

### **my_loans View:**
#### a) Design (Relational Algebra)

We want to join the `loans` and `posts` tables to return, for the current user, all loan records (even if there is no associated post). In relational algebra:

$$
\pi_{\text{loan\_attributes, post\_attributes}} \left( \sigma_{\text{user\_id} = \text{current\_user}} \left( \text{loans⟕posts} \right) \right)
$$

Here, ⟕ denotes a **left outer join**.

#### b) Implementation (SQL)
```sql

CREATE OR REPLACE VIEW my_loans AS
  SELECT l.signature,
         l.user_id,
         l.stopdate,
         l.town,
         l.province,
         l.type,
         l.time,
         l.return,
         p.post_date,
         p.text,
         p.likes,
         p.dislikes
  FROM loans l
  LEFT JOIN posts p 
    ON l.signature = p.signature 
   AND l.user_id = p.user_id 
   AND l.stopdate = p.stopdate
  WHERE l.user_id = foundicu_user.get_current_user();

```
> **Since only the post text is updatable, we must handle updates using an INSTEAD OF trigger:**

```sql

CREATE OR REPLACE TRIGGER trg_my_loans_update
INSTEAD OF UPDATE OF text ON my_loans
FOR EACH ROW
DECLARE
  v_exists NUMBER;
BEGIN
  -- Check if a post record already exists for this loan
  SELECT COUNT(*) INTO v_exists 
  FROM posts 
  WHERE signature = :NEW.signature 
    AND user_id = :NEW.user_id 
    AND stopdate = :NEW.stopdate;
  
  IF v_exists = 0 THEN
    -- Insert new post record
    INSERT INTO posts (signature, user_id, stopdate, post_date, text, likes, dislikes)
    VALUES (:NEW.signature, :NEW.user_id, :NEW.stopdate, SYSDATE, :NEW.text, 0, 0);
  ELSE
    -- Update the existing post record, setting post_date to current date/time
    UPDATE posts 
    SET text = :NEW.text, post_date = SYSDATE
    WHERE signature = :NEW.signature 
      AND user_id = :NEW.user_id 
      AND stopdate = :NEW.stopdate;
  END IF;
END;
/

```
#### c) Tests
```sql
-- To Query the View:
EXEC foundicu_user.set_current_user('9994309942');
SELECT * FROM my_data;
-- To test the trigger effect
    EXEC foundicu_user.set_current_user('9994309942');

    -- Update a loan’s post (whether it exists or not):
    UPDATE my_loans
    SET text = 'This book was amazing!'
    WHERE signature = 'VF987'
    AND stopdate = TO_DATE('23-NOV-2024','DD-MON-YYYY');

    -- Verify result:
    SELECT * FROM posts
    WHERE signature = 'VF987'
    AND user_id = '9994309942'
    AND stopdate = TO_DATE('23-NOV-2024','DD-MON-YYYY');

```
- **View Test Output**
    - This should return the loans that have user_id = 9994309942.
    - <img src="2.png" height="550" width="600">

- **Trigger Test Output**
    - The INSTEAD OF trigger should fire—checking if a corresponding record exists in POSTS, then inserting or updating it as necessary.
    - <img src="view2.png" height="350" width="800">
### **my_reservations View:**
#### a) Design (Relational Algebra)
Assuming reservations are stored in the `loans` table with type 'R' (for Reservation), the view selects those records for the current user:

$$
\pi_{\text{loan\_attributes}} \left( \sigma_{\text{user\_id} = \text{current\_user} \land \text{type} = 'R'} (\text{loans}) \right)
$$

#### b) Implementation (SQL)
```sql
CREATE OR REPLACE VIEW my_reservations AS
  SELECT signature,
         user_id,
         stopdate,
         town,
         province,
         type,
         time,
         return
  FROM LOANS
  WHERE user_id = foundicu_user.get_current_user()
    AND type = 'R';
```

> **If the business rule requires that only reservations for which a copy (or any copy of the same ISBN) is available for the next 14 days may be inserted, we must enforce this with a trigger because the view itself won’t check that. So we create trigger that checks for copy availability before inserting a reservation.**

**Insertion**
```sql
CREATE OR REPLACE TRIGGER trg_my_reservations_insert
INSTEAD OF INSERT ON my_reservations
FOR EACH ROW
DECLARE
   v_available NUMBER;
BEGIN
   -- Check if the copy (identified by the given signature) is available.
   -- We assume that the reservation row contains a signature that links to a copy.
   SELECT COUNT(*) INTO v_available
   FROM COPIES c
   WHERE c.signature = :NEW.signature
     AND c.deregistered IS NULL
     AND NOT EXISTS (
         SELECT 1 FROM LOANS l
         WHERE l.signature = c.signature
           AND l.return IS NULL
           AND l.stopdate BETWEEN :NEW.stopdate AND :NEW.stopdate + 14
     );
   
   IF v_available = 0 THEN
      RAISE_APPLICATION_ERROR(-20002, 'No available copy for reservation in the specified date range.');
   ELSE
      INSERT INTO LOANS (signature, user_id, stopdate, town, province, type, time, return)
      VALUES (:NEW.signature,
              :NEW.user_id,
              :NEW.stopdate,
              :NEW.town,
              :NEW.province,
              'R',  -- type is set to 'R' for reservation
              :NEW.time,
              :NEW.return);
   END IF;
END;
/
```
**Updation**
```sql
CREATE OR REPLACE TRIGGER trg_my_reservations_update
INSTEAD OF UPDATE OF stopdate ON my_reservations
FOR EACH ROW
BEGIN
   UPDATE LOANS
   SET stopdate = :NEW.stopdate
   WHERE signature = :OLD.signature
     AND user_id = :OLD.user_id
     AND stopdate = :OLD.stopdate
     AND type = 'R';
END;
/
```
**Deletion**
```sql
CREATE OR REPLACE TRIGGER trg_my_reservations_delete
INSTEAD OF DELETE ON my_reservations
FOR EACH ROW
DECLARE
  v_is_loaned NUMBER;
BEGIN
  -- Suppose you must check that the book is not yet loaned
  SELECT COUNT(*) INTO v_is_loaned
  FROM loans
  WHERE signature = :OLD.signature
    AND type = 'L'
    AND return IS NULL
    AND stopdate BETWEEN :OLD.stopdate AND :OLD.stopdate + 14;

  IF v_is_loaned > 0 THEN
    RAISE_APPLICATION_ERROR(-20002, 'Cannot delete reservation; the book has already been loaned.');
  ELSE
    DELETE FROM LOANS
    WHERE signature = :OLD.signature
      AND user_id = :OLD.user_id
      AND stopdate = :OLD.stopdate
      AND type = 'R';
  END IF;
END;
/
/
```
#### c) Tests
```sql
EXEC foundicu_user.set_current_user('USER000001');
-- To test the view
SELECT * FROM my_reservations;
-- To test insertion trigger
INSERT INTO my_reservations (
  signature, user_id, stopdate, town, province, type, time, return
)
VALUES (
  'PD137', '9994309942',
  TO_DATE('22-NOV-2024','DD-MON-YYYY'),
  'Sotomelones de las Caballerizas', 'Albacete',
  'R', 0, NULL
);

COMMIT;

-- To test updation trigger

UPDATE my_reservations
SET stopdate = TO_DATE('23-NOV-2024','DD-MON-YYYY')
WHERE signature = 'PD137'
  AND stopdate = TO_DATE('22-NOV-2024','DD-MON-YYYY');

--- This will satisfy the FK_LOANS_SERVICES constraint.
INSERT INTO services (  town, province, bus, taskdate, passport) VALUES (  'Sotomelones de las Caballerizas', 'Albacete',  'BUS-017', TO_DATE('23-NOV-2024', 'DD-MON-YYYY'),  'ESP>>101010101100');

SELECT * FROM my_reservations WHERE signature = 'PD137';

-- To test deletion trigger

DELETE FROM my_reservations
WHERE signature = 'PD137'
  AND stopdate = TO_DATE('23-NOV-2024','DD-MON-YYYY');

```

- Outputs
    - Insertion

    <img src="insert.png" height="350" width="800">

    - Deletion

    <img src="delete.png" height="350" width="800">

    - Updation

    <img src="update.png" height="350" width="800">

---

# Explicitly required Triggers

## Triggers Implementation

### **B. When the condition of a copy is set to ‘deteriorated’, the ‘deregistration date’ is automatically set to 'current date and time’.**
- Design
- **Objective**: Automatically set `DEREGISTERED` to SYSDATE when `CONDITION` is updated to 'D'.  
- **Tables Involved**: `copies`.  
- **Trigger Logic**:  
  - Before updating `CONDITION`, check if the new value is 'D' and the old value is not 'D'.  
  - Set `DEREGISTERED` to the current date only when the condition transitions to 'D'.  

- Implementation 
```sql
CREATE OR REPLACE TRIGGER set_deregistration_date
BEFORE UPDATE OF CONDITION ON copies
FOR EACH ROW
BEGIN
    IF :NEW.CONDITION = 'D' AND (:OLD.CONDITION != 'D' OR :OLD.CONDITION IS NULL) THEN
        :NEW.DEREGISTERED := SYSDATE;
    END IF;
END;
/
```

- Test:  
```sql
-- Setup: Insert a Copy with Condition 'G'
INSERT INTO editions (ISBN, TITLE, AUTHOR, LANGUAGE, NATIONAL_LIB_ID) 
VALUES ('ISBN_TEST', 'Test Book', 'Test Author', 'Spanish', 'NL_TEST');

INSERT INTO copies (SIGNATURE, ISBN, CONDITION) 
VALUES ('STEST', 'ISBN_TEST', 'G');

-- Test 1: Update to 'D' (Set DEREGISTERED)
UPDATE copies SET CONDITION = 'D' WHERE SIGNATURE = 'STEST';
SELECT SIGNATURE, CONDITION, DEREGISTERED FROM copies WHERE SIGNATURE = 'STEST'; -- DEREGISTERED should be SYSDATE

-- Test 2: Update to 'W' (No Change)
UPDATE copies SET CONDITION = 'W' WHERE SIGNATURE = 'STEST';
SELECT DEREGISTERED FROM copies WHERE SIGNATURE = 'STEST'; -- DEREGISTERED remains unchanged

-- Test 3: Update Back to 'D' (Update DEREGISTERED Again)
UPDATE copies SET CONDITION = 'D' WHERE SIGNATURE = 'STEST';
SELECT DEREGISTERED FROM copies WHERE SIGNATURE = 'STEST'; -- DEREGISTERED updates to new SYSDATE
```

 - Output
 
<img src="4b.png" height="450" width="500">

### **C. Create a ‘history table’ of both users and loans (not views, but another two tables). When a user is removed, create a history record of that user, and move all their loans to the history loans.**

- Design:
    - This document provides the complete implementation for the library database history feature, including design rationale, SQL code, and testing approach. The system creates history tables for both users and loans, ensuring that when a user is removed from the system, their information is preserved along with all their associated loan records.
- Implementation:
### History Tables Creation

```sql
-- Create history table for users
CREATE TABLE history_users (
    USER_ID            CHAR(10),
    ID_CARD            CHAR(17) NOT NULL,
    NAME               VARCHAR2(80) NOT NULL,
    SURNAME1           VARCHAR2(80) NOT NULL,
    SURNAME2           VARCHAR2(80),
    BIRTHDATE          DATE NOT NULL,
    TOWN               VARCHAR2(50) NOT NULL,
    PROVINCE           VARCHAR2(22) NOT NULL,
    ADDRESS            VARCHAR2(150) NOT NULL,
    EMAIL              VARCHAR2(100),
    PHONE              NUMBER(9) NOT NULL,
    TYPE               CHAR(1) NOT NULL,
    BAN_UP2            DATE,
    DELETION_DATE      DATE DEFAULT SYSDATE NOT NULL,
    CONSTRAINT pk_history_users PRIMARY KEY(USER_ID)
);

-- Create history table for loans
CREATE TABLE history_loans (
    SIGNATURE          CHAR(5),
    USER_ID            CHAR(10),
    STOPDATE           DATE,
    TOWN               VARCHAR2(50) NOT NULL,
    PROVINCE           VARCHAR2(22) NOT NULL,
    TYPE               CHAR(1) NOT NULL,
    TIME               NUMBER(5) NOT NULL,
    RETURN             DATE,
    MOVED_TO_HISTORY   DATE DEFAULT SYSDATE NOT NULL,
    CONSTRAINT pk_history_loans PRIMARY KEY(SIGNATURE, USER_ID, STOPDATE)
);
```
### User Deletion Trigger

```sql
CREATE OR REPLACE TRIGGER trg_user_delete
BEFORE DELETE ON users
FOR EACH ROW
DECLARE
    v_posts_count NUMBER;
BEGIN
    INSERT INTO history_users (
        USER_ID, ID_CARD, NAME, SURNAME1, SURNAME2, BIRTHDATE,
        TOWN, PROVINCE, ADDRESS, EMAIL, PHONE, TYPE, BAN_UP2
    ) VALUES (
        :OLD.USER_ID, :OLD.ID_CARD, :OLD.NAME, :OLD.SURNAME1, :OLD.SURNAME2, :OLD.BIRTHDATE,
        :OLD.TOWN, :OLD.PROVINCE, :OLD.ADDRESS, :OLD.EMAIL, :OLD.PHONE, :OLD.TYPE, :OLD.BAN_UP2
    );
    
    SELECT COUNT(*) INTO v_posts_count
    FROM posts p
    WHERE p.USER_ID = :OLD.USER_ID;
    
    IF v_posts_count > 0 THEN
        DELETE FROM posts
        WHERE USER_ID = :OLD.USER_ID;
    END IF;
    
    INSERT INTO history_loans (
        SIGNATURE, USER_ID, STOPDATE, TOWN, PROVINCE, TYPE, TIME, RETURN
    )
    SELECT 
        SIGNATURE, USER_ID, STOPDATE, TOWN, PROVINCE, TYPE, TIME, RETURN
    FROM loans
    WHERE USER_ID = :OLD.USER_ID;
    
    DELETE FROM loans
    WHERE USER_ID = :OLD.USER_ID;
END;
/
```

### Delete User Procedure

```sql
CREATE OR REPLACE PROCEDURE delete_user(p_user_id IN CHAR)
AS
BEGIN
    DELETE FROM users
    WHERE USER_ID = p_user_id;
    
    COMMIT;
    
    DBMS_OUTPUT.PUT_LINE('User ' || p_user_id || ' has been deleted and moved to history.');
EXCEPTION
    WHEN NO_DATA_FOUND THEN
        DBMS_OUTPUT.PUT_LINE('User ' || p_user_id || ' not found.');
    WHEN OTHERS THEN
        ROLLBACK;
        DBMS_OUTPUT.PUT_LINE('Error deleting user: ' || SQLERRM);
END delete_user;
/
```

- Test:

### Test 1: Deleting a User

```sql
SET SERVEROUTPUT ON;
EXEC delete_user('USER000001');
```

Expected output:
```
User USER000001 has been deleted and moved to history.
```

Verify in history table:

```sql
SELECT USER_ID, NAME || ' ' || SURNAME1 AS NAME, 
       TO_CHAR(DELETION_DATE, 'DD-MM-YYYY HH24:MI:SS') AS DELETED_ON 
FROM history_users;
```

Sample result:

| USER_ID    | NAME           | DELETED_ON           |
|------------|----------------|----------------------|
| USER000001 | Test User      | 04-04-2025 16:22:22  |


### Test 2: Deleting Another User

```sql
SELECT * FROM loans WHERE user_id = '8631945440';

EXEC delete_user('8631945440');
```

Sample results:

**history_users**

| USER_ID    | NAME              | DELETED_ON           |
|------------|-------------------|----------------------|
| 8631945440 | Alejandro Ochoa   | 04-04-2025 16:38:39  |

**history_loans**

```sql
SELECT * FROM history_loans WHERE user_id = '8631945440';
```

| SIGNATURE | USER_ID    | STOPDATE   | TOWN              | PROVINCE    | TYPE | TIME | RETURN     | MOVED_TO_HISTORY |
|-----------|------------|------------|-------------------|-------------|------|------|------------|------------------|
| NG009     | 8631945440 | 21-NOV-24  | Prados del Lago   | Ciudad Real | L    | 670  | 05-DEC-24  | 04-APR-25        |


### D. **Add a column ‘reads’ to Books. When a book is loaned, update the number of reads.**
- Design:
    - We add a new column reads to the books table, which tracks how many times a book has been loaned. The trg_update_book_reads trigger automatically increments this counter whenever a new loan record (type = ‘L’) is inserted into the loans table.
    - New Column:
        - ```sql ALTER TABLE books ADD (reads NUMBER DEFAULT 0); ```
        - This adds a numeric column reads to books. Any existing rows start with reads = 0.
    - Logic:
        - The trigger fires AFTER INSERT on loans.
        - It checks whether the new row is a loan (type = ‘L’) rather than a reservation (type = ‘R’).
        - If so, it finds the isbn from copies (using the signature), then retrieves the corresponding title and author from editions.
        - Finally, it increments reads by 1 for the matching row in books.
- Implementation:
    ```sql 

        CREATE OR REPLACE TRIGGER update_book_reads
        AFTER INSERT ON loans
        FOR EACH ROW
        DECLARE
            v_title books.title%TYPE;
            v_author books.author%TYPE;
        BEGIN
            -- Get the title and author for the borrowed book using the signature
            SELECT e.title, e.author INTO v_title, v_author
            FROM copies c
            JOIN editions e ON c.isbn = e.isbn
            WHERE c.signature = :NEW.signature;
            
            -- Update the reads count for the book
            UPDATE books
            SET reads = reads + 1
            WHERE title = v_title AND author = v_author;
            
            -- For debugging purposes
            -- DBMS_OUTPUT.PUT_LINE('Updated reads count for book: ' || v_title || ' by ' || v_author);
        EXCEPTION
            WHEN NO_DATA_FOUND THEN
                DBMS_OUTPUT.PUT_LINE('Error: Could not find book information for signature: ' || :NEW.signature);
            WHEN OTHERS THEN
                DBMS_OUTPUT.PUT_LINE('Error updating reads count: ' || SQLERRM);
        END;
        /
    ```

- Test:
    ```sql
    -- First, delete the existing loan record
    DELETE FROM loans
    WHERE signature = 'NK117'
    AND user_id = '5681444606' 
    AND stopdate = TO_DATE('19-NOV-24', 'DD-MON-YY');

    -- Check the current reads value for the book by Hendrik Groen
    SELECT title, author, reads FROM books
    WHERE author = 'Groen, Hendrik'
    AND title = 'Intentos de sacarle algo a la vida ::  el diario de Hendrik Groen, de ochenta y tres aÃ±os y cuarto.';

    -- Reinsert the loan to trigger the update_book_reads trigger
    INSERT INTO loans (signature, user_id, stopdate, town, province, type, time, return)
    VALUES ('NK117', 
            '5681444606', 
            TO_DATE('19-NOV-24', 'DD-MON-YY'),
            'Nava de la Torre',
            'Soria',
            'L',
            600,
            TO_DATE('03-DEC-24', 'DD-MON-YY'));

    -- Check if the reads count was incremented
    SELECT title, author, reads FROM books
    WHERE author = 'Groen, Hendrik'
    AND title = 'Intentos de sacarle algo a la vida ::  el diario de Hendrik Groen, de ochenta y tres aÃ±os y cuarto.';

    ```
- Output

    <img src="4D.png" height="400" width="700">
---



# Concluding Remarks

- The queries and triggers worked well and covered real-life use cases like managing loans, reservations, and tracking book reads.

- The results matched expectations during testing, and the logic handled edge cases smoothly (like banned users or unavailable copies).

- Creating the views and packages made the system feel more user-friendly and organized.

- We spent around 18–20 hours overall—learning a lot about relational algebra, SQL joins, grouping, and PL/SQL triggers.

- For future labs, a visual schema and sample data would help a lot. Splitting the work into smaller parts could also make things easier.

---